# Setup

In [1]:
from consav.runtools import write_numba_config
write_numba_config(disable=0,threads=20,threading_layer='omp')

In [2]:
import time
import numpy as np

%load_ext autoreload
%autoreload 2
%load_ext line_profiler

from TwoAssetModelCont import TwoAssetModelContClass

# UMFPACK

## Without Howard improvement

In [3]:
UMFPACK_noHIS = TwoAssetModelContClass(name='UMFPACK',solmethod='UMFPACK')
UMFPACK_noHIS.par.start_HIS = np.inf
UMFPACK_noHIS.solve(do_print=False)
UMFPACK_noHIS.solve()
print('')
UMFPACK_noHIS.show_moments()

Grids created in 0.1 secs
Solution prepared in 0.1 secs
Solving HJB:
    1: 84.1540116083472753
    2: 36.4077721627507245
    3: 30.0339429262052171
    4: 29.2756855009812398
    5: 19.8659501420693374
    6: 18.3467315209349096
    7: 14.4465947306913449
    8: 12.1943889342600045
    9: 10.3557107522892551
 converged in 9.0 secs in iteration 98

Solving KFE:
    1: 3073.0138171345465707
    2: 4082.4265798398600964
    3: 973.1253575407322387
    4: 643.8266931913253757
    5: 443.0377089858584441
    6: 285.2705299967237806
    7: 227.0352880793320764
    8: 147.4253950664306103
    9: 120.5199295933314261
  100: 0.0003002165500448
 converged in 3.3 secs secs in iteration 149

Equilibrium objects:
 rb: 0.0050
 ra: 0.0142
 w: 1.814
 Pi: 0.171
 capital-labor discrepancy: 0.00240791
 profit discrepancy: 0.01786609

Aggregates:
 GDP: 1.740
 capital-labor ratio: 28.098
 capital-output ratio: 2.336
 bond-output ratio: 0.233

Fractions:
 poor HtM: 0.103 [0.103, 0.146]
 wealthy HtM: 0.186

## With Howard improvement

In [4]:
UMFPACK = TwoAssetModelContClass(name='UMFPACK',solmethod='UMFPACK')
UMFPACK.solve(do_print=False)
UMFPACK.solve()
print('')
UMFPACK.show_moments()

Grids created in 0.1 secs
Solution prepared in 0.1 secs
Solving HJB:
    1: 84.1540116083472753
    2: 36.4077721627507245
    3: 30.0339429262052171
    4: 61.9424646878300820
    5: 36.3830590774077649
    6: 22.9675880962306991
    7: 14.2673877293000970
    8: 8.8447807774702198
    9: 5.5025591312347615
 converged in 5.4 secs in iteration 38

Solving KFE:
    1: 3073.0131534679812830
    2: 4082.4381678185036435
    3: 973.1202905865966386
    4: 643.8314789443302288
    5: 443.0379078356850187
    6: 285.2738276914169546
    7: 227.0367155099625052
    8: 147.4276629460373442
    9: 120.5213490492380970
  100: 0.0003002320113410
 converged in 3.2 secs secs in iteration 149

Equilibrium objects:
 rb: 0.0050
 ra: 0.0142
 w: 1.814
 Pi: 0.171
 capital-labor discrepancy: 0.00240905
 profit discrepancy: 0.01786575

Aggregates:
 GDP: 1.740
 capital-labor ratio: 28.098
 capital-output ratio: 2.336
 bond-output ratio: 0.233

Fractions:
 poor HtM: 0.103 [0.103, 0.146]
 wealthy HtM: 0.186 [

## Line profiler

In [5]:
from solve import solve_HJB, solve_KFE, howard_improvement_steps
%lprun -f solve_HJB -f solve_KFE -f howard_improvement_steps UMFPACK.solve(do_print=False)

Timer unit: 3.42135e-07 s

Total time: 2.05531 s
File: C:\Users\gmf123.UNICPH\Documents\Repositories\HANKPY_old\solve.py
Function: howard_improvement_steps at line 456

Line #      Hits         Time  Per Hit   % Time  Line Contents
   456                                           def howard_improvement_steps(par,sol,ast,solmethod,cppfile):
   457                                               """ take howard improvement steps """
   458                                           
   459        90        689.0      7.7      0.0      for _ in range(par.maxiter_HIS):
   460                                           
   461                                                   # a. create RHS
   462        60      14929.0    248.8      0.2          v_prev_HIS = sol.v.copy()
   463        60    2317844.0  38630.7     38.6          create_RHS_HJB(par,sol,ast,v_prev_HIS)
   464                                                   
   465                                                   # b. solve
   

# Comparison with scipy

In [6]:
scipy = TwoAssetModelContClass(name='scipy',solmethod='scipy')
scipy.solve()
print('')
scipy.show_moments()

Grids created in 2.6 secs
Solution prepared in 2.7 secs
Solving HJB:
    1: 84.1540116083473748
    2: 36.4077721627265305
    3: 30.0339429261163495
    4: 61.9424646871130804
    5: 36.3830590770564299
    6: 22.9675880963969803
    7: 14.2673877293798768
    8: 8.8447807774929572
    9: 5.5025591312693507
 converged in 35.4 secs in iteration 38

Solving KFE:
    1: 3073.0131534599090628
    2: 4082.4381678616900899
    3: 973.1202905717004796
    4: 643.8314789632777320
    5: 443.0379078319656401
    6: 285.2738277018106601
    7: 227.0367155097164868
    8: 147.4276629518758455
    9: 120.5213490499152158
  100: 0.0003002320109999
 converged in 18.8 secs secs in iteration 149

Equilibrium objects:
 rb: 0.0050
 ra: 0.0142
 w: 1.814
 Pi: 0.171
 capital-labor discrepancy: 0.00240905
 profit discrepancy: 0.01786575

Aggregates:
 GDP: 1.740
 capital-labor ratio: 28.098
 capital-output ratio: 2.336
 bond-output ratio: 0.233

Fractions:
 poor HtM: 0.103 [0.103, 0.146]
 wealthy HtM: 0.186

In [7]:
models = [UMFPACK,scipy]
for model in models:
    print(model.name)
    print(f"K/Y: {model.moms['KY']:.8f}")
    print(f"B/Y: {model.moms['BY']:.8f}")
    print('')

UMFPACK
K/Y: 2.33640449
B/Y: 0.23253223

scipy
K/Y: 2.33640449
B/Y: 0.23253223

